# Fix customer dashboard data

In [2]:
import pandas as pd
import numpy as np
import boto3
import time
import sqlalchemy

Download the data from redshift to dataframe named "data"

In [ ]:

#Enter Here the table of the new merchant data instead of playground.lightricks_data

QUERY = """
SELECT *
FROM playground.chegg_acquiring 
"""

def extract_data_from_result(result):
    columns = []
    rows = []

    for column in result["ColumnMetadata"]:
        columns.append(column["name"])

    for result_row in result["Records"]:
        row_values = []
        for result_column in result_row:
            # Handle None values
            row_values.append(list(result_column.values())[0] if result_column else None)
        rows.append(row_values)

    return columns, rows

if __name__ == "__main__":
    redshift_client = boto3.client("redshift-data", region_name="us-east-1")
    
    # Execute the query
    response = redshift_client.execute_statement(
        Database="analytics",
        DbUser='eedobounce',
        ClusterIdentifier="analytics-redshift",
        Sql=QUERY,
        
    )
    print(f"Query Response: {response}")
    
    query_id = response["Id"]
    query_status = "STARTED"
    
    # Check query status in a loop
    while query_status in ["STARTED", "SUBMITTED", "PICKED", "WIP"]:
        time.sleep(5)  # Wait for 5 seconds before checking the status again
        query_state = redshift_client.describe_statement(Id=query_id)
        query_status = query_state["Status"]
        print(f"Query State: {query_state}")
    
    # Check the final status of the query
    if query_status == "FINISHED":
        print("Query finished successfully.")
        try:
            results = redshift_client.get_statement_result(Id=query_id)
        except redshift_client.exceptions.ResourceNotFoundException:
            print("Resource not found.")
            results = {"ColumnMetadata": [], "Records": []}
        
        # Extract data and create DataFrame
        columns, rows = extract_data_from_result(result=results)
        data = pd.DataFrame(rows, columns=columns)
        print("DataFrame created successfully.")
        print(data.head())
    else:
        print(f"Query did not finish successfully. Final status: {query_status}")

In [ ]:
# Download exchange rate table from Redshift 
QUERY = """
SELECT *
FROM dms_analyst.exchangerate
"""

def extract_data_from_result(result):
    columns = []
    rows = []

    for column in result["ColumnMetadata"]:
        columns.append(column["name"])

    for result_row in result["Records"]:
        row_values = []
        for result_column in result_row:
            # Handle None values
            row_values.append(list(result_column.values())[0] if result_column else None)
        rows.append(row_values)

    return columns, rows

if __name__ == "__main__":
    redshift_client = boto3.client("redshift-data", region_name="us-east-1")
    
    # Execute the query to fetch the exchange rate table
    response = redshift_client.execute_statement(
        Database="analytics",
        DbUser='eedobounce',
        ClusterIdentifier="analytics-redshift",
        Sql=QUERY,
    )
    print(f"Query Response: {response}")
    
    query_id = response["Id"]
    query_status = "STARTED"
    
    # Check query status in a loop
    while query_status in ["STARTED", "SUBMITTED", "PICKED", "WIP"]:
        time.sleep(5)  # Wait for 5 seconds before checking the status again
        query_state = redshift_client.describe_statement(Id=query_id)
        query_status = query_state["Status"]
        print(f"Query State: {query_state}")
    
    # Check the final status of the query
    if query_status == "FINISHED":
        print("Query finished successfully.")
        try:
            results = redshift_client.get_statement_result(Id=query_id)
        except redshift_client.exceptions.ResourceNotFoundException:
            print("Resource not found.")
            results = {"ColumnMetadata": [], "Records": []}
        
        # Extract data and create DataFrame for the exchange rate table
        columns, rows = extract_data_from_result(result=results)
        exchange_rate_table = pd.DataFrame(rows, columns=columns)
        print("Exchange Rate DataFrame created successfully.")
        print(exchange_rate_table.head())
    else:
        print(f"Query did not finish successfully. Final status: {query_status}")


exchange_rate_table['date'] = pd.to_datetime(exchange_rate_table['date'])


# Data Cleaning

In [7]:
def clean_data(data, columns_mapping, unique_columns, sort_data):
    
    # Mapping of columns
    
    for k, v in columns_mapping.items():
        if v in data.columns:
            data.rename(columns={v: k}, inplace=True)
    
    #Remove duplicates based on unique_columns and duplicate_strategy
    sorted_by = unique_columns + sort_data
    data = data.sort_values(by=sorted_by)
    data.drop_duplicates(subset=unique_columns, keep='first', inplace=True)
    
    # Status mapping Left is the status in the data and right is the status we want to map to
    status_mapping = {
        'Approved': 'Approved',
        'Declined': 'Declined',
        'Failed': 'Declined',  
        'Paid': 'Approved',
        'Success': 'Approved'
        # Add more mappings as needed
    }
    
    #status mapping    
    data['status'] = data['status'].apply(lambda x: status_mapping.get(x, x))

    # Declined reason mapping
    # Define conditions and corresponding values as a dictionary
    regex_values_dict = {
        r'.*insufficient.*': 'insufficient funds',   
        r'payment.*complete': 'payment complete',
        r'do.*not.*honor': 'do not honor',
        r'transaction.*not.*allowed': 'transaction not allowed',
        r'stripe.*block': 'blocked by Stripe',
        r'.*did.*not.*return': 'no further details',  
        r'block.*lists': 'block lists rules',
        r'.*expired.*card': 'expired card',
        r'.*try.*again.*later': 'try again later',
        r'.*invalid.*account': 'invalid account',
        r'.*invalid.*pin.*': 'invalid pin',
        r'.*invalid.*cvc.*': 'invalid cvc',
        r'.*invalid.*amount.*': 'invalid amount',
        r'incorrect.*number': 'incorrect number',
        r'.*incorrect.*cvc.*': 'incorrect cvc',
        r'.*blocked.*by.*merchant.*rule': 'blocked by merchant rule',
        r'pickup.*card': 'pickup_card',
        r'processing.*error': 'processing error',
        r'stolen.*card': 'stolen card',
        r'lost.*card': 'lost card',
        r'.*fraud.*': 'Processor Declined - Fraud Suspected',
        r'limit.*exceeded': 'insufficient funds',
        r'approved': 'Approved',
        r'reenter.*transaction': 'reenter_transaction',
        r'Life cycle.*': 'cannot authorize at this time life cycle',
        r'82.*Policy': 'expired card',
        r'.*Insufficient.*': 'insufficient funds',
        r'card_velocity_exceeded': 'card velocity exceeded',
        r'withdrawal_count_limit_exceeded': 'withdrawal count limit exceeded',
        r'incorrect_number': 'incorrect number',
        r'do_not_honor': 'do not honor',
        r'call_issuer': 'call issuer',
        r'transaction_not_allowed': 'transaction not allowed',
        r'revocation_of_authorization': 'revocation of authorization',
        r'revocation_of_all_authorization': 'revocation of all authorization',
        r'reenter_transaction': 'reenter transaction',
        r'service_not_allowed': 'service not allowed',
        r'generic_decline': 'generic decline',
        r'no_action_taken': 'no action taken',
        r'stop_payment_order': 'stop payment order',
        r'try_again_later': 'try again later',
        r'processing_error': 'processing error',
        r'do_not_try_again': 'do not try again',
        r'issuer_not_available': 'issuer not available',
        r'.*invalid_number.*': 'invalid number',
        r'highest_risk_level': 'highest risk level',
        r'NULL': 'stripe blocked payment',
        r'previously_declined_do_not_retry': 'stripe blocked payment',
        r'requested_block_on_incorrect_cvc': 'requested block on incorrect cvc',
        r'.*expired_card.*': 'expired card',
        r'invalid_expiry_month': 'invalid expiry month',
        r'invalid_expiry_year': 'invalid expiry year',
        r'.*stolen.*card.*': 'stolen card',
        r'.*lost_card.*': 'lost card',
        r'.*pickup_card.*': 'pickup card',
        r'blocklist': 'blocklist',
        r'merchant_blacklist': 'merchant blacklist',
        r'rule': 'merchant rule',
        r'.*incorrect_pin.*': 'incorrect pin',
        r'pin_try_exceeded': 'pin try exceeded',
        r'authentication_required': 'authentication required',
        r'approve_with_id': 'approve with id',
        r'security_violation': 'security violation',
        r'elevated_risk_level': 'elevated risk level',
        r'restricted_card': 'card restriction',
        r'fraudulent': 'stripe fraud',
        r'currency_not_supported': 'currency not supported',
        r'card_not_supported': 'card not supported',
        r'duplicate_transaction': 'duplicate transaction',
        r'incorrect_zip': 'incorrect zip',
        r'new_account_information_available': 'new account information available',
        r'not_permitted': 'not permitted',
        r'testmode_decline': 'stripe test decline',
        r'offline_pin_required': 'offline pin required',
        r'online_or_offline_pin_required': 'online or offline pin required',
        r'1000': 'approved',
        r'1001': 'approved',
        r'1002': 'approved',
        r'1003': 'approved',
        r'1004': 'approved',
        r'2001': 'insufficient funds',
        r'2002': 'limit exceeded',
        r'2003': 'incorrect number',
        r'2005': 'incorrect number',
        r'2000': 'do not honor',
        r'2044': 'declined call issuer',
        r'2046': 'declined',
        r'2013': 'stolen card',
        r'2007': 'invalid account',
        r'2004': 'expired card',
        r'2047': 'pickup card',
        r'2010': 'incorrect cvc',
        r'2021': 'security violation',
        r'2020': 'violation',
        r'2057': 'card restriction',
        r'3000': 'try again later',
        r'2026': 'invalid merchant id',
        r'2014': 'processor declined fraud suspected',
        r'2019': 'invalid transaction',
        r'2048': 'invalid amount',
        r'2009': 'no such issuer',
        r'2016': 'duplicate transaction',
        r'2037': 'already reversed',
        r'2017': 'cardholder stopped billing',
        r'2018': 'cardholder stopped billing',
        r'2056': 'transaction amount exceeded division amount',
        r'2041': 'call issuer',
        r'2038': 'processor declined',
        r'2015': 'transaction not allowed',
        r'2034': 'no action taken',
        r'2043': 'do not try again',
        r'2008': 'card account length error',
        r'2022': 'updated cardholder available',
        r'2006': 'invalid expiration date',
        r'2053': 'lost or stolen',
        r'2012': 'lost card',
        r'2102': 'incorrect pin',
        r'2103': 'pin try exceeded',
        r'2105': 'cannot authorize at this time life cycle',
        r'2106': 'cannot authorize at this time policy',
        r'2107': 'card not activated',
        r'.*Deny.*': 'declined',
        r'.*Payment complete.*': 'Approved',
        r'.*85 : No reason to decline a request for account number verification.*': 'Approved'
    }

    #Can do a Lamda function instead
    # Use regular expressions to apply conditions and assign values to the new column based on the dictionary
    for regex, value in regex_values_dict.items():
        data.loc[data['decline_reason'].str.contains(regex, case=False, na=False, regex=True), 'decline_reason'] = value

    # Loop over unique decline reasons
    for reason in data['decline_reason'].unique():
        # Check if the reason is not found in the values dictionary
        if reason not in regex_values_dict.values():
            # Assign 'other' to rows where the decline reason matches
            data.loc[data['decline_reason'] == reason, 'decline_reason'] = 'other'
    
    
    # Ensure the 'currency' column is of type string and convert to uppercase
    data['currency'] = data['currency'].astype(str).str.upper()
    
    return data

# Data Validation

In [8]:
def data_validation(data, unique_columns):
 
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)
    
    # Check for duplicate transaction IDs
    duplicate_transactions = data.duplicated(subset= unique_columns).sum()
    print(f"Duplicate transaction by unique columns: {duplicate_transactions}")
    print("")
    print("")
    status_counts = data['status'].value_counts(normalize=True)
    approved_count = status_counts.get('Approved', 0)
    declined_count = status_counts.get('Declined', 0)

    # Check if the percentage of "Approved" or "Declined" is less than 10%
    print( "Check the percentage of 'Approved' or 'Declined' ")
    if approved_count < 0.1:
        print("######### Warning: Approved transactions are less than 10% of the total data.###############")
    if declined_count < 0.1:
        print("######### Warning: Declined transactions are less than 10% of the total data.###############")

    print(data['status'].value_counts())
    print("")
    print("")

    # Check for 'created' in column names and if it's a timestamp
    print("Check timestamp if exists in the data:")
    try:
        # Attempt to convert to datetime
        data['created_at'] = pd.to_datetime(data['created_at'])
    except ValueError as e:
        print(f"Created at is now a possible time_stamp: {e}")    
    if pd.api.types.is_datetime64_any_dtype(data['created_at']):
        min_date = data['created_at'].min()
        max_date = data['created_at'].max()
        print("created_at is a timestamp column.")
        print("Minimum date:", min_date)
        print("Maximum date:", max_date)
        print("Date difference (days):", (max_date - min_date).days)
    else:
        print("No timestamp column containing 'created' found.")

    print("")
    print("")
    print("Checks for US currency:")
    currency_counts = data['currency'].value_counts(normalize=True) 
    usd_counts = currency_counts.get('USD', 0)
    if usd_counts < 0.1:
        print("Warning: Missing data from US")
    else:
        print("US data is exist.")
    print(data['currency'].value_counts(normalize=True))
    print("")
    print("")
    # Check which currencies are missing in the exchange rate table
    exchange_currencies = exchange_rate_table['currency'].unique()
    data_currencies = data['currency'].unique()
    missing_currencies = [currency for currency in data_currencies if currency not in exchange_currencies]
    print("Missing Currencies from table:")
    print(missing_currencies)
    
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns') 
    pd.reset_option('display.expand_frame_repr', False)
   

# Feature Engineering 

In [ ]:
def feature_engineering(data, grouped_by, order_by, fixed_exchange_rates):
    #Adding root transactions
    if order_by = 'status'
        status_priority = {'Declined': 1, 'Approved': 2}

        # Apply custom sorting within each group and assign rank
        data['retry_number'] = (
            data.assign(status_priority=data['status'].map(status_priority))
            .sort_values(by=grouped_by + ['status_priority', order_by])
            .groupby(grouped_by)
            .cumcount() + 1
        )

    # Drop the helper column used for sorting
    data.drop(columns='status_priority', inplace=True)
    else:
        data['retry_number'] = data.groupby(grouped_by)[order_by].rank(method='first')
    
    # Adding  last retry decision 
    grouped_data = data.groupby(grouped_by)['status'].apply(lambda x: x.isin(['Success', 'Approved']).max()).reset_index(name='last_retry_decision')

    # Merge the aggregated result back to the original DataFrame based on the grouping columns
    data = data.merge(grouped_data, on=grouped_by, how='left')
    
    #last retry decline reason for renewals
    # Determine the max retry number for each transaction_id
    max_retry_per_transaction = data.groupby(grouped_by)['retry_number'].max().reset_index()

    # Merge with original dataframe to get corresponding decline reason for max retry
    merged_data = pd.merge(data, max_retry_per_transaction, on=grouped_by + ['retry_number'], how='inner')
    # data['last_retry_decline_reason'] = merged_data['decline_reason']
    # Create a dictionary mapping transaction_id to the corresponding decline_reason for the max retry number
    transaction_id_to_decline_reason = dict(zip(merged_data['transaction_id'], merged_data['decline_reason']))

    # Map the transaction_id in data to the corresponding decline_reason using the dictionary
    data['last_retry_decline_reason'] = data['transaction_id'].map(transaction_id_to_decline_reason)
    
    # Add USD amount column
    data = pd.merge(data, exchange_rate_table, how='left', left_on=['currency', 'created_at'], right_on=['currency', 'date'])

    # Apply lambda function to calculate usd_amount
    data['usd_amount'] = data.apply(
        lambda row: round(row['amount'] * (row['rate'] if pd.notna(row['rate']) else fixed_exchange_rates.get(row['currency'], None)), 1) if fixed_exchange_rates.get(row['currency'], None) or pd.notna(row['rate']) else None,
        axis=1
    )
    columns_to_remove = ['id', 'rate', 'date', 'created_at_y', 'updated_at']
    data.drop(columns=columns_to_remove, inplace=True)
    return data

# Fill the data with the instruction below 

In [82]:
columns_mapping = {
    'transaction_id': 'transaction_id',
    'decline_reason': 'processor_response_text',
    'created_at': 'invoice_date',
    'amount': 'amount',
    'currency': 'iso_currency_code',
    'buyer_id': 'user_id',  # - Customer email/ subscription_id (For identifying self retries)
    'retry': '', #If they have column fo retry 
    'status': 'status', # Success/Failed/Approved/Declined etc..
    'type': '', # Transaction type checkout/renewal
    'last_retry_decision': '', # - Decision of the last retry (Approve/Decline) will add after
    'usd_amount': '', # - Will add after
    'retry_number': '', # - Will add after
    'last_retry_decline_reason': '' # if it's a renewal will add after
}


#Here write the columns needs to be unique with no duplicates
unique_columns = ['transaction_id', 'created_at', 'amount']  # Enter column name who should be unique
sort_data = []
data_clean = clean_data(data, columns_mapping, unique_columns, sort_data) 



In [83]:
data_validation(data_clean, unique_columns)


Duplicate transaction by unique columns: 0


Check the percentage of 'Approved' or 'Declined' 
status
Approved     474403
Declined     267987
Cancelled       534
Name: count, dtype: int64


Check timestamp if exists in the data:
created_at is a timestamp column.
Minimum date: 2024-03-01 00:00:00
Maximum date: 2024-03-31 00:00:00
Date difference (days): 30


Checks for US currency:
US data is exist.
currency
USD    0.849422
CAD    0.038443
AUD    0.024023
KRW    0.018112
GBP    0.016630
TRY    0.011495
PHP    0.009230
ZAR    0.008471
HKD    0.005614
SAR    0.005403
MXN    0.004174
SGD    0.002806
AED    0.002380
MYR    0.002240
IDR    0.001470
INR    0.000052
EUR    0.000035
Name: proportion, dtype: float64


Missing Currencies from table:
['USD', 'MYR', 'HKD', 'SAR', 'AED', 'IDR', 'PHP', 'TRY', 'SGD', 'ZAR', 'MXN']


In [66]:
# Define the columns to group by and order by in order to determine retry number
grouped_by = ['user_id', 'created_at', 'amount'] 

order_by = 'status'
fixed_exchange_rates = {
    'USD': 1,
    'BGN': 0.56,
    'DKK': 0.15,
    'NZD': 0.63,
    'RSD': 0.0092,
    'PLN': 0.25,
    'BDT': 0.0094,
    'SGD': 0.76,
    'TRY': 0.034,
    'KES': 0.0064,
    'NOK': 0.098,
    'ZAR': 0.055,
    'PEN': 0.27,
    'ISK': 0.0074,
    'AMD': 0.0025,
    'DZD': 0.0074,
    'USD': 1,
    'AUD': 0.68,
    'HUF': 0.0029,
    'DOP': 0.017,
    'BRL': 0.21,
    'MXN': 0.059,
    'CHF': 1.19,
    'UAH': 0.026,
    'RUB': 0.011,
    'AED': 0.27,
    'CRC': 0.0019,
    'THB': 0.029,
    'EUR': 1.10,
    'MYR': 0.22,
    'COP': 0.00026,
    'VND': 0.000041,
    'GBP': 1.27,
    'ARS': 0.0012,
    'GEL': 0.37,
    'PKR': 0.0036,
    'SAR': 0.27,
    'ILS': 0.28,
    'INR': 0.012,
    'HKD': 0.13,
    'SEK': 0.099,
    'JMD': 0.0065,
    'PHP': 0.018,
    'CLP': 0.0011,
    'JPY': 0.0071,
    'EGP': 0.032,
    'KRW': 0.00077,
    'RON': 0.22,
    'TWD': 0.032,
    'TND': 0.32,
    'NGN': 0.0011,
    'IDR': 0.000064,
    'CAD': 0.76,
    'PHP': 0.017
    
}
fix_data =  feature_engineering(data_clean, grouped_by, order_by, fixed_exchange_rates)
fix_data.head()